In [71]:
import os 

In [72]:
%pwd

'd:\\vevik\\Text-Summarizer'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\vevik\\Text-Summarizer'

In [73]:
from dataclasses import dataclass
from pathlib import Path
from typing import Union

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Union[str, Path]

In [74]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
            config = self.config.data_transformation

            create_directories([config.root_dir])

            data_transformation_config = DataTransformationConfig(
                   root_dir=config.root_dir,
                   data_path=config.data_path,
                   tokenizer_name=config.tokenizer_name,
            )

            return data_transformation_config

In [ ]:
import os 
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name, model_max_length=1024)
        self.tokenizer.pad_token = self.tokenizer.eos_token


    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True, padding="max_length", return_tensors="pt")

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True, padding="max_length", return_tensors="pt")

        return{
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
            
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True,)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [91]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-04-06 03:40:18,770: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-06 03:40:18,771: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-06 03:40:18,772: INFO: common: created directory at: artifacts]
[2025-04-06 03:40:18,772: INFO: common: created directory at: artifacts/data_transformation]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\khadh\anaconda3\envs\textS\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\khadh\AppData\Local\Temp\ipykernel_17340\451100680.py", line 7, in <module>
    raise e
  File "C:\Users\khadh\AppData\Local\Temp\ipykernel_17340\451100680.py", line 4, in <module>
    data_transformation = DataTransformation(config=data_transformation_config)
  File "C:\Users\khadh\AppData\Local\Temp\ipykernel_17340\307671132.py", line 5, in __init__
    self.tokenizer = PegasusTokenizer.from_pretrained(config.tokenizer_name, model_max_length=1024)
  File "c:\Users\khadh\anaconda3\envs\textS\lib\site-packages\transformers\utils\import_utils.py", line 1849, in __getattribute__
  File "c:\Users\khadh\anaconda3\envs\textS\lib\site-packages\transformers\utils\import_utils.py", line 1837, in requires_backends
ImportError: 
PegasusTokenizer requires the SentencePiece l